# Talktorial 11 (part C)

# CADD web services that can be used via a Python API

__Developed at AG Volkamer, Charité__

Dr. Jaime Rodríguez-Guerra

## Aim of this talktorial

> This is part C of the "Online webservices" talktorial:
>
> - 11a. Querying KLIFS & PubChem for potential kinase inhibitors
> - 11b. Docking the candidates against the target obtained in 11a
> - __11c. Assessing the results and comparing against known data__

After obtaining input structures and docking them, we will assess whether the results are any good.

## Learning goals

### Theory

- Protein-ligand interactions
- False positives in docking

### Practical

- Visualize the results
- Run automated analysis

### Discussion

Pending.

### Quiz

Pending.

***

# Theory

## Protein-ligand interactions

Pending

## False positives

Pending

***

# Practical

## Visualize the results

Use `nglview` for that! It's compatible with `PDBQT` files out of the box.

In [119]:
import pandas as pd
import time
import nglview as nv
from glob import glob
from ipywidgets import AppLayout, Button, Layout, Select, HTML

In [120]:
def create_viewer(protein, ligands, affinities):
    """
    Create a nglview widget with the protein and all the ligands labeled by affinities
    """
    v = nv.show_file(protein)
    label_kwargs = dict(labelType="text", sele="@0", showBackground=True, backgroundColor="black")
    for ligand, affinity in zip(ligands, affinities):
        c = v.add_component(ligand)
        c.add_label(labelText=[str(affinity)], **label_kwargs)
    return v

def split_pdbqt(path):
    """
    Split a multimodel PDBQT into separate files.
    """
    files = []
    with open(path) as f:
        lines = []
        i = 0
        for line in f:
            lines.append(line)
            if line.strip() == 'ENDMDL':
                fn = f'data/results.{i}.pdbqt'
                with open(fn, 'w') as o:
                    o.write(''.join(lines))
                files.append(fn)
                i += 1
                lines = []
    return files

def parse_output(out):
    """
    Create a DataFrame out of the Vina output file
    """
    with open(out) as f:
        data = []
        for line in f:
            if line.startswith('-----+'):
                line = next(f)
                while line.split()[0].isdigit():
                    index, *floats = line.split()
                    data.append([int(index)] + list(map(float, floats)))
                    line = next(f)
    return pd.DataFrame.from_records(data, 
                                     columns=['Mode', 'Affinity (kcal/mol)', 'RMSD (l.b.)', 'RMSD (u.b.)'], 
                                     exclude=['Mode'])

In [107]:
def _on_selection_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        v.hide(list(range(1,10)))
        component = getattr(v, f"component_{change['new']}")
        component.show()
        component.center()

In [121]:
def show_docking(protein, ligands, vina_output):
    # Split the multi PDBQT ligand file into separate files
    ligands_files = split_pdbqt(ligands)
    # Retrieve affinities
    affinities = parse_output(vina_output)['Affinity (kcal/mol)']
    
    # Create viewer widget
    v = show_docking_results(protein, ligands_files, affinities)
    # Protein is loaded first (component 0), then all ligands
    # Hide all but the protein and first ligand
    v.hide(list(range(2,len(ligands_files) + 1)))
    v.component_1.center() # center on the first ligand
    
    # Create selection widget
    selector = Select(options=[(f"#{i} {aff} kcal/mol", i) for (i, aff) in enumerate(affinities, 1)],
                      description="",  rows=10, layout=Layout(width="auto"))
    
    # This is the event handler - action taken when the user clicks on the select box
    def _on_selection_change(change):
        if change['name'] == 'value' and (change['new'] != change['old']):
            v.hide(list(range(1,len(ligands_files) + 1))  # Hide all ligands
            component = getattr(v, f"component_{change['new']}")
            component.show()  # Display the selected one
            component.center()  # Zoom view
    selector.observe(_on_selection_change)
                                
    # Arrange GUI elements
    return AppLayout(left_sidebar=selector, center=v, pane_widths=[1, 6, 1])

In [122]:
show_docking("data/protein.mol2", "data/results.pdbqt", "data/vina.out")

AppLayout(children=(Select(layout=Layout(grid_area='left-sidebar', width='auto'), options=(('#1 -7.2 kcal/mol'…